# Llama 3.2 fine tuning with size

2024-12-14 22:07

Fine tuning Llama instruct 3.2 with the size data set using unsloth. Using a larger test split because the training datset is very small which lends itself to some memorization. Results are good although not 100% accurate. I had several browser crashed halting the process indefinitely, possibly due to resource exhaustion.

In [1]:
!apt-get install build-essential -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


In [2]:
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!pip install sacrebleu
!pip install pytest-playwright
!playwright install
!pip install matplotlib
!pip install pillow
!pip install torchvision
!pip install lpips

!playwright install-deps  

!pip install -U numpy
!pip install tensorboard

Found existing installation: unsloth 2024.12.4
Uninstalling unsloth-2024.12.4:
  Successfully uninstalled unsloth-2024.12.4
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-dh29swu4/unsloth_84dc871a0807405f9fe2ab5efc066557
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-dh29swu4/unsloth_84dc871a0807405f9fe2ab5efc066557
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.4-py3-none-any.whl size=173746 sha256=154671529f02db87f889b3571f1609746b757956dc59d404045927541975a0ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-im9d1h30/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
Installing dependencies...
Hit:1 http://arch

In [3]:
import os
import numpy as np
import pandas as pd

import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel
from datasets import Dataset
from unsloth import is_bfloat16_supported

# Saving model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Warnings
import warnings
warnings.filterwarnings("ignore")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
max_seq_length = 5020
def load_model():
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/Llama-3.2-1B-bnb-4bit",
        max_seq_length=max_seq_length,
        load_in_4bit=True,
        dtype=None,
    )
    
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,
        lora_alpha=16,
        lora_dropout=0,
        target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
        use_rslora=True,
        use_gradient_checkpointing="unsloth",
        random_state = 32,
        loftq_config = None,
    )
    return model, tokenizer

In [5]:
def create_trainer(model, tokenizer, training_data, max_steps):
    return SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=training_data,
        dataset_text_field="text",
        max_seq_length=max_seq_length,
        dataset_num_proc=2,
        packing=True,
        args=TrainingArguments(
            learning_rate=3e-4,
            lr_scheduler_type="linear",
            per_device_train_batch_size=16,
            gradient_accumulation_steps=8,
            num_train_epochs=1,
            fp16=not is_bfloat16_supported(),
            bf16=is_bfloat16_supported(),
            logging_steps=1,
            max_steps=max_steps,
            optim="adamw_8bit",
            weight_decay=0.01,
            warmup_steps=10,
            output_dir="output",
            seed=0,
        ),
    )

In [6]:
import numpy as np
from utils.similarity import calculate_metrics
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import torch

log_dir = 'output/runs'

def add_image_to_tensorboard(name, step, img_path):
    image = Image.open(img_path)
    image = image.convert('RGB')
    image_array = np.array(image)
    image_tensor = torch.from_numpy(image_array)
    image_tensor = image_tensor.permute(2, 0, 1)
    image_tensor = image_tensor.float() / 255.0
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_image(name, image_tensor, step)
    
def add_text_to_tensorboard(name, step, text):
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_text(name, text, step)

def postprocess_text(preds, labels):
    preds = [pred.strip().replace('<unk>', '') for pred in preds]
    labels = [[label.strip().replace('<unk>', '')] for label in labels]

    return preds, labels

def compute_metrics(decoded_predictions, decoded_labels, steps):
    similarity_scores = []
    perceptual_losses = []
    index = 1
    
    for prediction, label in zip(decoded_predictions, decoded_labels):
        prediction = prediction.replace(tokenizer.eos_token, '')
        
        add_text_to_tensorboard(f'valid_{index}_label_text', steps, label[0])
        add_text_to_tensorboard(f'valid_{index}_prediction_text', steps, prediction)
        
        metrics = calculate_metrics(prediction, label)
        
        if metrics is not None:
            similarity_scores.append(metrics['similarity'])
            perceptual_losses.append(metrics['perceptual_loss'])
            
            add_image_to_tensorboard(f'valid_{index}_expectation', steps, metrics['expected_screenshot_path'])
            add_image_to_tensorboard(f'valid_{index}_prediction', steps, metrics['predicted_screenshot_path'])
        
        index += 1

    results = {
        "similarity": float(np.mean(similarity_scores)),
        "perceptual_loss": float(np.mean(perceptual_losses)),
    }
    
    writer = SummaryWriter(log_dir=log_dir)
    writer.add_scalar('similarity', results['similarity'], steps)
    writer.add_scalar('perceptual_loss', results['perceptual_loss'], steps)
    
    print("Similarity:", results['similarity'])
    print("Perceptual loss:", results['perceptual_loss'])

    return results

def test_prediction(model, data, steps):
    answers = []
    labels = []
    print("Generating predictions...")
    for row in data:
        inputs = tokenizer(
        [
            data_prompt.format(
                #instructions
                row['svg'],
                #answer
                "",
            )
        ], return_tensors = "pt").to("cuda")
        
        outputs = model.generate(**inputs, max_new_tokens = 5020, use_cache = True)
        answer = tokenizer.batch_decode(outputs)
        answers.append(answer[0].split("### Response:")[-1])
        labels.append(row['html'])

    print("Computing metrics...")
    compute_metrics(answers, labels, steps)

In [7]:
!rm -rf output

!apt install zip -y
!rm -rf data-rb-size
!mkdir -p data-rb-size
!wget "https://www.dropbox.com/scl/fi/q2qp4fyi37qkksgvyjco6/data-rb-size.zip?rlkey=37cpoqqpcykwgksqnqforij8u&dl=1" -O model.zip
!unzip model.zip -d data-rb-size

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
--2024-12-14 18:47:04--  https://www.dropbox.com/scl/fi/q2qp4fyi37qkksgvyjco6/data-rb-size.zip?rlkey=37cpoqqpcykwgksqnqforij8u&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.13.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.13.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb7a16b8aa192bb8fe0074595a5.dl.dropboxusercontent.com/cd/0/inline/CgTN5Al0yJHZTOBxOiRTghCC5IFEBlFT9KngGywjFGmdEplkprrFj-qmCKYt3wQ0fsP8w0jJmNUOdgI4-SSyRGLjcJZIpnqhc3wYTE7bAcQ0TdQkUC67gbCMzMT7Lv162NI/file?dl=1# [following]
--2024-12-14 18:47:06--  https://ucb7a16b8aa192bb8fe0074595a5.dl.dropboxusercontent.com/cd/0/inline/CgTN5Al0yJHZTOBxOiRTghCC5IFEBlFT9KngGywjFGmdEplkprrFj-qmCKYt3wQ0fsP8w0jJmNUOdgI4-SSyRGLjcJZIpnqhc

In [8]:
!mkdir -p data-rb-validate

In [9]:
from datasets import load_from_disk
dataset = load_from_disk('data-rb-size')
#dataset = dataset.train_test_split(test_size=4/len(dataset))
dataset = dataset.train_test_split(0.10)

dataset

DatasetDict({
    train: Dataset({
        features: ['svg', 'html'],
        num_rows: 89
    })
    test: Dataset({
        features: ['svg', 'html'],
        num_rows: 10
    })
})

In [10]:
model, tokenizer = load_model()

data_prompt = """Your job is to take an SVG file of a web design and convert it into a pixel-perfect HTML and CSS markup and stylesheet.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompt(examples):
    inputs       = examples["svg"]
    outputs      = examples["html"]
    texts = []
    for input_, output in zip(inputs, outputs):
        text = data_prompt.format(input_, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

training_data = dataset.map(formatting_prompt, batched=True)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA H100 NVL. Max memory: 93.003 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2024.12.4 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


Map:   0%|          | 0/89 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [11]:
training_data

DatasetDict({
    train: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 89
    })
    test: Dataset({
        features: ['svg', 'html', 'text'],
        num_rows: 10
    })
})

In [12]:
import torch
from tqdm import tqdm

resume = False
for steps in tqdm(range(0, 1701, 100)):
    print(f"Steps: {steps}")

    if steps > 0:
        trainer = create_trainer(model, tokenizer, training_data['train'], steps)
        if resume:
            trainer.train(resume_from_checkpoint=True)
        else:
            trainer.train()
            resume = True
        
    model = FastLanguageModel.for_inference(model)

    results = test_prediction(model, training_data['test'], steps)

    if results is not None and results['perceptual_loss'] == 0.0:
        break

    model = FastLanguageModel.for_training(model)

    

  0%|          | 0/18 [00:00<?, ?it/s]

Steps: 0
Generating predictions...
Computing metrics...


  6%|▌         | 1/18 [15:00<4:15:05, 900.35s/it]

Similarity: 0.5932220205105841
Perceptual loss: 0.3038145639002323
Steps: 100


Generating train split: 0 examples [00:00, ? examples/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 6 | Num Epochs = 1,700
O^O/ \_/ \    Batch size per device = 16 | Gradient Accumulation steps = 8
\        /    Total batch size = 128 | Total steps = 1,700
 "-____-"     Number of trainable parameters = 11,272,192


Step,Training Loss
1601,0.001600
1602,0.001600
1603,0.001600
1604,0.001600
1605,0.001600
1606,0.001600
1607,0.001600
1608,0.001600
1609,0.001600
1610,0.001600


Generating predictions...
Computing metrics...


100%|██████████| 18/18 [1:10:38<00:00, 235.46s/it]

Similarity: 0.9572119904070859
Perceptual loss: 0.06702561471611261


In [19]:
test_index = 4
text = training_data['test'][test_index]['svg']
model = FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    data_prompt.format(
        #instructions
        text,
        #answer
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 5020, use_cache = True)
answer=tokenizer.batch_decode(outputs)
answer = answer[0].split("### Response:")[-1]

print(training_data['test'][test_index]['html'])
print("Answer of the question is:", answer)

<body><div></div></body>

<style>


        body {
            margin: 0;
        }

        div {
            background-color: black;
            position: absolute;
            height: 100vh;
            width: 12%;
            top: 0;
            left: 0;
        }


</style>
Answer of the question is: 
<body><div></div></body>

<style>


        body {
            margin: 0;
        }

        div {
            background-color: black;
            position: absolute;
            height: 100vh;
            width: 11%;
            top: 0;
            left: 0;
        }


</style><|end_of_text|>


In [16]:
test_prediction(model, training_data['test'], steps)

Generating predictions...
Computing metrics...
Similarity: 0.9572119904070859
Perceptual loss: 0.06702561471611261
